In [1]:
import gc
import json
import warnings

import torch
import chess
import chess.pgn
from tqdm import tqdm

warnings.filterwarnings("ignore")

def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

torch.manual_seed(42)

# Cleanup
clear_memory()

In [2]:
# Load PGN games
from pathlib import Path

pgn_file = Path("games.pgn")
MAX_GAMES = 100 

games = []
with open(pgn_file, "r", encoding="utf-8") as pgn:
    with tqdm(total=MAX_GAMES, desc="Reading PGN (games)") as pbar:
        for _ in range(MAX_GAMES):
            try:
                game = chess.pgn.read_game(pgn)
                if game is None:  # end of file
                    break
                games.append(game)
            except Exception as e:
                print(f"Skipping corrupted game: {e}")
                continue
            pbar.update(1)

print(f"Loaded {len(games)} games (limit = {MAX_GAMES}).")

Reading PGN (games): 100%|██████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 400.96it/s]

Loaded 100 games (limit = 100).


In [ ]:
import os
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM

HF_TOKEN = ""
if HF_TOKEN:
    login(token=HF_TOKEN)
else:
    print("No Hugging Face token found. Make sure to set HF_TOKEN in your environment.")

# Model choice
model_name = "google/gemma-3-1b-it"

# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32
print(f"Loading model in {dtype}...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=dtype,
    device_map="auto" 
)

print("Model and tokenizer loaded successfully.")

Loading tokenizer...
Loading model in torch.bfloat16...
Model and tokenizer loaded successfully.


In [4]:
# Single-prompt generation
def safe_generate(prompt, model, tokenizer, max_new_tokens=64):
    """
    Generate commentary for a single chess move safely.
    Ensures robustness against bad prompts, GPU OOM, or decoding errors.
    """
    if not prompt.strip():
        return "[Invalid prompt]"

    device = next(model.parameters()).device

    try:
        # Tokenize
        inputs = tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(device)

        # Inference
        with torch.inference_mode():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                top_p=0.9,
                temperature=0.8,
                pad_token_id=tokenizer.eos_token_id
            )

        # Decode
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Strip prompt from response
        if "Provide a chess commentary" in text:
            commentary = text.split("Provide a chess commentary")[-1]
        else:
            commentary = text

        return commentary.strip() or "[Empty generation]"

    except RuntimeError as e: 
        if "CUDA" in str(e):
            torch.cuda.empty_cache()
        return f"[RuntimeError: {str(e)[:50]}...]"
    except Exception as e:
        return f"[Error: {str(e)[:50]}...]"

In [6]:
def safe_generate_batch(prompts, model, tokenizer, max_new_tokens=64, batch_size=4):
    """
    Generate commentary for multiple prompts in small batches safely.
    Returns a list of outputs aligned with input prompts.
    """
    results = []
    device = next(model.parameters()).device

    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i + batch_size]

        # Replace empty prompts with a placeholder
        cleaned_batch = [p if p.strip() else "[Invalid prompt]" for p in batch]

        try:
            inputs = tokenizer(
                cleaned_batch,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=512
            ).to(device)

            with torch.inference_mode():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    do_sample=True,
                    top_p=0.9,
                    temperature=0.8,
                    pad_token_id=tokenizer.eos_token_id
                )

            # Decode each sequence
            for text in tokenizer.batch_decode(outputs, skip_special_tokens=True):
                if "Provide a chess commentary" in text:
                    commentary = text.split("Provide a chess commentary")[-1]
                else:
                    commentary = text
                commentary = commentary.strip() or "[Empty generation]"
                results.append(commentary)

        except RuntimeError as e:
            if "CUDA" in str(e):
                torch.cuda.empty_cache()
            results.extend([f"[RuntimeError: {str(e)[:50]}...]"] * len(batch))
        except Exception as e:
            results.extend([f"[Error: {str(e)[:50]}...]"] * len(batch))

    assert len(results) == len(prompts), "Mismatch in outputs and inputs"
    return results

In [7]:
BATCH_SIZE = 32
MAX_NEW_TOKENS = 32
all_commentaries = []

total_moves = sum(1 for g in games for _ in g.mainline_moves())

with tqdm(total=total_moves, desc="Generating commentary", unit="move") as pbar:
    for game_idx, game in enumerate(games, 1):
        board = game.board()
        game_commentary = []

        batch_prompts, batch_moves = [], []

        for move in game.mainline_moves():
            san_move = board.san(move)
            fen = board.fen()  
            prompt = (
                f"Chess board FEN: {fen}\n"
                f"Move: {san_move}\n"
                f"Provide a chess commentary explaining the move:"
            )

            batch_prompts.append(prompt)
            batch_moves.append(san_move)
            board.push(move)

            # Process batch when full
            if len(batch_prompts) >= BATCH_SIZE:
                comments = safe_generate_batch(
                    batch_prompts, model, tokenizer,
                    max_new_tokens=MAX_NEW_TOKENS, 
                    batch_size=BATCH_SIZE
                )
                game_commentary.extend(zip(batch_moves, comments))
                pbar.update(len(batch_prompts))  # update per move
                batch_prompts, batch_moves = [], []

        # Flush leftovers
        if batch_prompts:
            comments = safe_generate_batch(
                batch_prompts, model, tokenizer,
                max_new_tokens=MAX_NEW_TOKENS, 
                batch_size=len(batch_prompts)
            )
            game_commentary.extend(zip(batch_moves, comments))
            pbar.update(len(batch_prompts))

        all_commentaries.append(game_commentary)

        if game_idx % 50 == 0:
            print(f"Processed {game_idx}/{len(games)} games ({total_moves} moves total).")

print(f"Generated commentary for {len(games)} games ({total_moves} moves).")

Generating commentary:  54%|████████████████████████████▌                        | 3334/6193 [08:44<08:07,  5.87move/s]

Processed 50/100 games (6193 moves total).


Generating commentary: 100%|█████████████████████████████████████████████████████| 6193/6193 [15:39<00:00,  6.59move/s]

Processed 100/100 games (6193 moves total).
Generated commentary for 100 games (6193 moves).


In [8]:
output_file = "chess_commentary.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for game_idx, game_data in enumerate(all_commentaries, 1):
        for move, commentary in game_data:
            record = {"game": game_idx, "move": move, "commentary": commentary}
            f.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"Saved commentary to {output_file}")

Saved commentary to chess_commentary.jsonl


In [14]:
import re

def clean_commentary(comment, move):
    """
    Normalize commentary into at most 2 concise sentences.
    """
    # Remove weird formatting and commentary markers
    text = re.sub(r"```.*?```", "", comment, flags=re.DOTALL)
    text = text.replace("**Commentary:**", "").replace("Commentary:", "").strip()

    # Split into sentences
    sentences = re.split(r'(?<=[.!?])\s+', text)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 8]  # filter too-short junk

    # Keep max 2 sentences
    if len(sentences) >= 2:
        return f"{sentences[0]} {sentences[1]}"
    elif len(sentences) == 1:
        return sentences[0]
    else:
        return f"{move} is a natural developing move, supporting control of the board."


import random

def summarize_game(game_commentary):
    """
    Generate a natural-sounding conclusion summary of the game (at least 5 sentences),
    with randomized phrasing like a coach or commentator.
    """
    moves = [m for m, _ in game_commentary]
    n_moves = len(moves)
    first_move = moves[0] if moves else "N/A"
    last_move = moves[-1] if moves else "N/A"

    joined_comments = " ".join(c for _, c in game_commentary).lower()
    if "attack" in joined_comments:
        style = "attacking play"
    elif "defense" in joined_comments:
        style = "defensive play"
    elif "center" in joined_comments:
        style = "control of the center"
    else:
        style = "a mix of strategic and tactical play"

    # Sentence pools
    openings = [
        f"This game stretched across {n_moves} moves, kicking off with {first_move}.",
        f"Over {n_moves} moves, starting with {first_move}, the players set the stage for a fascinating struggle.",
        f"From the very first move {first_move}, the battle promised to be an interesting one."
    ]

    styles = [
        f"Both sides leaned heavily on {style}, which shaped the early flow of the game.",
        f"The character of the game was marked by {style}, especially in the opening phase.",
        f"{style.capitalize()} was a recurring theme throughout the struggle."
    ]

    middlegames = [
        "The middlegame saw careful maneuvering and exchanges as both searched for an edge.",
        "Once the opening was complete, the players entered a middlegame full of tension and sharp choices.",
        "The heart of the game was fought in the middlegame, with momentum shifting back and forth."
    ]

    finals = [
        f"The game eventually concluded with {last_move}, which brought the encounter to its close.",
        f"In the end, {last_move} marked the finishing touch of this long battle.",
        f"The struggle wrapped up with {last_move}, a fitting final move."
    ]

    conclusions = [
        "Overall, it was a lively contest full of lessons for improving players.",
        "Altogether, the game showed creativity, resilience, and fighting spirit from both sides.",
        "This was the kind of encounter that highlights the richness of chess."
    ]

    # Build randomized summary
    summary = [
        random.choice(openings),
        random.choice(styles),
        random.choice(middlegames),
        random.choice(finals),
        random.choice(conclusions),
    ]

    return " ".join(summary)

import random
game_index = random.randint(0, len(all_commentaries) - 1)
sample_game = all_commentaries[game_index]

print(f"\n=== Cleaned Commentary for Random Game {game_index+1} ===\n")
for move_number, (move, comment) in enumerate(sample_game, 1):
    fixed_comment = clean_commentary(comment, move)
    print(f"{move_number:2d}. {move:<5} → {fixed_comment}")

print("\n=== Game Summary ===")
print(summarize_game(sample_game))


=== Cleaned Commentary for Random Game 99 ===

 1. d4    → explaining the move:

This is a solid opening move for White. It is a classic and reliable approach that leads to a controlled and dynamic position.
 2. d5    → explaining the move:
The move d5 is a common and principled opening move for White. It controls the center of the board and prepares to develop a piece.
 3. c4    → explaining the move:
The move c4 is a common opening move in chess. It develops a pawn to a central square, controlling the center of the board.
 4. Nf6   → explaining the move:

Nf6 is a common and strong opening move in chess. It controls the center and prepares for further development.
 5. Nc3   → explaining the move:

The move Nc3 is a common and principled opening move in chess. It develops a knight to a central square, controlling the d5 square and preparing to
 6. e6    → explaining the move:
This is a very common and solid opening move for White. It challenges Black's control of the center.
 7. a3  

In [12]:
# Find the shortest game (fewest moves in all_commentaries)
shortest_game = min(all_commentaries, key=len)
shortest_index = all_commentaries.index(shortest_game)

print(f"\n=== Cleaned Commentary for Shortest Game (Game {shortest_index+1}, {len(shortest_game)} moves) ===\n")
for move_number, (move, comment) in enumerate(shortest_game, 1):
    fixed_comment = clean_commentary(comment, move)
    print(f"{move_number:2d}. {move:<5} → {fixed_comment}")

print("\n=== Game Summary ===")
print(summarize_game(shortest_game))



=== Cleaned Commentary for Shortest Game (Game 97, 7 moves) ===

 1. b3    → explaining the move: Trong the chess game, the player has made a move to b3. This move is a common opening sequence, aiming to control the center of the board.
 2. g6    → explaining the move: Trong comment. **Comment:**
The move g6 is a good opening move for White.
 3. Bb2   → explaining the move: засесть в B1. The move Bb2 is a solid opening move for White.
 4. Nf6   → explaining the move: Trong the opening, the pawn is developed to a central square. The knight is placed in a good position to attack and defend.
 5. g4    → explaining the move: Trong comment, include the current board position, and the reasoning behind the move. ```
Rnk
N
p
p
p
p
 6. b6    → explaining the move: אות, and why it is a good move. ```
The move b6 is a good move, because it controls the center of the board and opens
 7. g5    → explaining the move:
The move g5 is a standard opening move in chess, developing a knight to a central,